# 🚀 Smart Money Tracker - Version edgartools

**✅ Connexions SEC EDGAR fonctionnelles via edgartools**

Ce notebook utilise `edgartools` pour résoudre les problèmes de User-Agent et parsing XML.

---

## 📦 Imports et Configuration

In [1]:
import sys
import os
from pathlib import Path

# Ajouter le répertoire racine au path
root_dir = Path.cwd()
if str(root_dir) not in sys.path:
    sys.path.insert(0, str(root_dir))

# Imports standards
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import warnings
warnings.filterwarnings('ignore')

# Imports pour visualisation
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

# Configuration pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("✅ Imports standards chargés")
print(f"📂 Répertoire de travail: {root_dir}")

✅ Imports standards chargés
📂 Répertoire de travail: c:\n8n-local-stack


In [3]:
# Import edgartools
from edgar import Company, set_identity

# Configurer l'identité SEC (requis)
set_identity("n8n-local-stack research@mtlstreetboy.com")

print("✅ edgartools configuré")
print("   User-Agent SEC: n8n-local-stack research@mtlstreetboy.com")
print("   Rate limiting: Automatique (géré par edgartools)")

✅ edgartools configuré
   User-Agent SEC: n8n-local-stack research@mtlstreetboy.com
   Rate limiting: Automatique (géré par edgartools)


In [13]:
# Import du nouvel analyzer avec edgartools
import importlib
import sys

# Force reload du module si déjà importé
if 'prod.analysis.edgar_smart_money_analyzer' in sys.modules:
    importlib.reload(sys.modules['prod.analysis.edgar_smart_money_analyzer'])

from prod.analysis.edgar_smart_money_analyzer import EdgarSmartMoneyAnalyzer
analyzer = EdgarSmartMoneyAnalyzer()

print("✅ EdgarSmartMoneyAnalyzer chargé (module rechargé)")
print("   Utilise: edgartools (moderne, rate limiting auto)")
print("   Form 4: ✅ Fonctionnel (parsing corrigé)")
print("   13F: ✅ Disponible")
print("   Political: ⚠️  Sources gratuites bloquées (GitHub/S3)")

INFO:edgar.core:Identity of the Edgar REST client set to [n8n-local-stack research@mtlstreetboy.com]
INFO:prod.analysis.edgar_smart_money_analyzer:EdgarSmartMoneyAnalyzer initialized with edgartools


✅ EdgarSmartMoneyAnalyzer chargé (module rechargé)
   Utilise: edgartools (moderne, rate limiting auto)
   Form 4: ✅ Fonctionnel (parsing corrigé)
   13F: ✅ Disponible
   Political: ⚠️  Sources gratuites bloquées (GitHub/S3)


In [6]:
# Configuration pour tests
TEST_CONFIG = {
    'sec_user_agent': 'n8n-local-stack research@example.com',
    'rate_limits': {
        'sec_edgar': 9,  # Pour respecter limite SEC
    },
    'thresholds': {
        'high_conviction_min_value': 100000,  # $100k minimum
    },
    'analysis_windows': {
        'political_cluster': 14,  # 14 jours
        'insider_cluster': 7,
    }
}

# Tickers de test (petite liste pour itération rapide)
TEST_TICKERS = ['NVDA', 'AAPL', 'TSLA', 'MSFT', 'GOOGL']

print("✅ Configuration chargée")
print(f"📊 Tickers de test: {TEST_TICKERS}")

✅ Configuration chargée
📊 Tickers de test: ['NVDA', 'AAPL', 'TSLA', 'MSFT', 'GOOGL']


## 🚀 Initialisation de l'Analyzer

---

# 📊 TEST 1: Transactions Politiques

Collecte des transactions du Congrès (Sénat + Chambre des Représentants)

In [9]:
# Collecter les transactions politiques (90 derniers jours)
print("🔄 Collection des transactions politiques en cours...\n")

political_df = analyzer.collect_political_trades(days_back=90)

print(f"\n✅ Collection terminée: {len(political_df)} transactions")

INFO:prod.analysis.edgar_smart_money_analyzer:💡 Consider using: Quiver Quant API, FMP API, or Capitol Trades


🔄 Collection des transactions politiques en cours...


✅ Collection terminée: 0 transactions


In [6]:
# Afficher un aperçu des données
if not political_df.empty:
    print("📋 APERÇU DES TRANSACTIONS POLITIQUES:\n")
    display(political_df.head(10))
    
    print(f"\n📊 STATISTIQUES:")
    print(f"   - Période: {political_df['transaction_date'].min()} → {political_df['transaction_date'].max()}")
    print(f"   - Politiciens uniques: {political_df['politician'].nunique()}")
    print(f"   - Tickers uniques: {political_df['ticker'].nunique()}")
    print(f"   - Sénat: {len(political_df[political_df['chamber'] == 'Senate'])}")
    print(f"   - Chambre: {len(political_df[political_df['chamber'] == 'House'])}")
else:
    print("⚠️ Aucune transaction politique trouvée")

⚠️ Aucune transaction politique trouvée


In [11]:
# Analyser les types de transactions
if not political_df.empty:
    print("📊 RÉPARTITION PAR TYPE:\n")
    type_counts = political_df['type'].value_counts()
    display(type_counts)
    
    # Graphique
    fig, ax = plt.subplots(figsize=(10, 5))
    type_counts.plot(kind='bar', ax=ax, color=['green' if 'buy' in t.lower() or 'purchase' in t.lower() else 'red' for t in type_counts.index])
    ax.set_title('Répartition des Types de Transactions Politiques', fontsize=14, fontweight='bold')
    ax.set_xlabel('Type de Transaction')
    ax.set_ylabel('Nombre')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

In [ ]:
# Top tickers les plus tradés par les politiciens
if not political_df.empty:
    print("🔥 TOP 10 TICKERS LES PLUS TRADÉS PAR LES POLITICIENS:\n")
    top_tickers = political_df['ticker'].value_counts().head(10)
    display(top_tickers.to_frame('Nombre de trades'))
    
    # Graphique
    fig, ax = plt.subplots(figsize=(10, 6))
    top_tickers.plot(kind='barh', ax=ax, color='steelblue')
    ax.set_title('Top 10 Tickers - Transactions Politiques', fontsize=14, fontweight='bold')
    ax.set_xlabel('Nombre de Transactions')
    ax.set_ylabel('Ticker')
    plt.tight_layout()
    plt.show()

---

# 🔍 TEST 2: Détection de Clusters Politiques

Identification des achats groupés (signal le plus fort)

In [7]:
# Détecter les clusters d'achats politiques
if not political_df.empty:
    print("🔍 Détection des clusters d'achats politiques...\n")
    
    clusters_df = analyzer.detect_political_clusters(
        political_df,
        window_days=14
    )
    
    print(f"✅ {len(clusters_df)} clusters détectés\n")
else:
    clusters_df = pd.DataFrame()
    print("⚠️ Pas de données politiques pour détecter des clusters")

⚠️ Pas de données politiques pour détecter des clusters


In [ ]:
# Afficher les clusters détectés
if not clusters_df.empty:
    print("🔥 CLUSTERS D'ACHATS POLITIQUES DÉTECTÉS:\n")
    display(clusters_df[['ticker', 'cluster_date', 'num_buyers', 'signal_strength', 'confidence_score', 'politicians']].head(15))
    
    # Statistiques
    print(f"\n📊 STATISTIQUES DES CLUSTERS:")
    print(f"   - Clusters très forts (🔥🔥🔥): {len(clusters_df[clusters_df['signal_strength'].str.contains('TRÈS FORT')])}")
    print(f"   - Clusters forts (🔥🔥): {len(clusters_df[clusters_df['signal_strength'].str.contains('FORT') & ~clusters_df['signal_strength'].str.contains('TRÈS')])}")
    print(f"   - Score de confiance moyen: {clusters_df['confidence_score'].mean():.1f}/100")
    print(f"   - Nombre moyen d'acheteurs par cluster: {clusters_df['num_buyers'].mean():.1f}")
else:
    print("⚠️ Aucun cluster détecté")

In [ ]:
# Visualisation des clusters par force de signal
if not clusters_df.empty:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Distribution des scores de confiance
    ax1.hist(clusters_df['confidence_score'], bins=20, color='steelblue', edgecolor='black', alpha=0.7)
    ax1.set_title('Distribution des Scores de Confiance', fontsize=12, fontweight='bold')
    ax1.set_xlabel('Score de Confiance')
    ax1.set_ylabel('Nombre de Clusters')
    ax1.axvline(clusters_df['confidence_score'].mean(), color='red', linestyle='--', label=f'Moyenne: {clusters_df["confidence_score"].mean():.1f}')
    ax1.legend()
    
    # Nombre d'acheteurs par cluster
    buyer_counts = clusters_df['num_buyers'].value_counts().sort_index()
    ax2.bar(buyer_counts.index, buyer_counts.values, color='coral', edgecolor='black', alpha=0.7)
    ax2.set_title('Distribution du Nombre d\'Acheteurs par Cluster', fontsize=12, fontweight='bold')
    ax2.set_xlabel('Nombre d\'Acheteurs')
    ax2.set_ylabel('Nombre de Clusters')
    ax2.set_xticks(buyer_counts.index)
    
    plt.tight_layout()
    plt.show()

---

# 👔 TEST 3: Transactions d'Initiés (Form 4)

Collecte des transactions d'initiés pour un ticker spécifique

In [14]:
# Choisir un ticker pour tester
TEST_TICKER = 'NVDA'  # Nvidia (souvent actif)

print(f"🔍 Collection des transactions d'initiés pour {TEST_TICKER}...\n")
print("⏳ Cela peut prendre 1-2 minutes (parsing XML depuis SEC EDGAR)\n")

insider_df = analyzer.collect_insider_trades(
    ticker=TEST_TICKER,
    days_back=90
)

print(f"\n✅ Collection terminée: {len(insider_df)} transactions")

INFO:prod.analysis.edgar_smart_money_analyzer:📥 Collecting Form 4 for NVDA via edgartools...
INFO:httpxthrottlecache.filecache.transport:cache_dir=C:\Users\palom\.edgar\_tcache
INFO:httpxthrottlecache.filecache.transport:No cache policy for data.sec.gov:///submissions/CIK0001045810.json, not retrieving from cache
INFO:httpxthrottlecache.ratelimiter:Making HTTP Request <Request('GET', 'https://data.sec.gov/submissions/CIK0001045810.json')>
INFO:httpx:HTTP Request: GET https://data.sec.gov/submissions/CIK0001045810.json "HTTP/1.1 200 OK"


🔍 Collection des transactions d'initiés pour NVDA...

⏳ Cela peut prendre 1-2 minutes (parsing XML depuis SEC EDGAR)



INFO:prod.analysis.edgar_smart_money_analyzer:   Company: NVIDIA CORP
INFO:httpxthrottlecache.filecache.transport:No cache policy for data.sec.gov:///submissions/CIK0001045810-submissions-001.json, not retrieving from cache
INFO:httpxthrottlecache.ratelimiter:Making HTTP Request <Request('GET', 'https://data.sec.gov/submissions/CIK0001045810-submissions-001.json')>
INFO:httpx:HTTP Request: GET https://data.sec.gov/submissions/CIK0001045810-submissions-001.json "HTTP/1.1 200 OK"
INFO:prod.analysis.edgar_smart_money_analyzer:   Found 20 Form 4 filings
INFO:httpxthrottlecache.controller:matched .*www\.sec\.gov, using value www.sec.gov: {'/submissions.*': 30, '/include/ticker\\.txt.*': 30, '/files/company_tickers\\.json.*': 30, '.*index/.*': 1800, '/Archives/edgar/data': True}
INFO:httpxthrottlecache.controller:/Archives/edgar/data/1045810/000119903925000015/0001199039-25-000015.txt matched /Archives/edgar/data, using value True
INFO:httpxthrottlecache.filecache.transport:Cache policy allo


✅ Collection terminée: 119 transactions


In [15]:
# Afficher les transactions d'initiés
if not insider_df.empty:
    print(f"📋 TRANSACTIONS D'INITIÉS - {TEST_TICKER}:\n")
    display(insider_df[['transaction_date', 'insider_name', 'role', 'transaction_code', 'type', 'shares', 'price_per_share', 'transaction_value']].head(10))
    
    print(f"\n📊 STATISTIQUES:")
    print(f"   - Période: {insider_df['transaction_date'].min()} → {insider_df['transaction_date'].max()}")
    print(f"   - Initiés uniques: {insider_df['insider_name'].nunique()}")
    print(f"   - Achats: {len(insider_df[insider_df['type'] == 'BUY'])}")
    print(f"   - Ventes: {len(insider_df[insider_df['type'] == 'SELL'])}")
    print(f"   - Valeur totale des transactions: ${insider_df['transaction_value'].sum():,.0f}")
else:
    print(f"⚠️ Aucune transaction d'initié trouvée pour {TEST_TICKER}")
    print("\n💡 Causes possibles:")
    print("   - Aucun Form 4 déposé dans les 90 derniers jours")
    print("   - Ticker non trouvé ou CIK invalide")
    print("   - Problème de connexion SEC EDGAR")

📋 TRANSACTIONS D'INITIÉS - NVDA:



,transaction_date,insider_name,role,transaction_code,type,shares,price_per_share,transaction_value
2,2025-12-19,Donald F Robertson Jr,Principal Accounting Officer,S,SELL,400,176.9075,7.076300e+04
3,2025-12-19,Donald F Robertson Jr,Principal Accounting Officer,S,SELL,1800,178.3511,3.210320e+05
4,2025-12-19,Donald F Robertson Jr,Principal Accounting Officer,S,SELL,6400,179.3552,1.147873e+06
5,2025-12-19,Donald F Robertson Jr,Principal Accounting Officer,S,SELL,14300,180.4451,2.580365e+06
6,2025-12-19,Donald F Robertson Jr,Principal Accounting Officer,S,SELL,1690,180.8837,3.056935e+05
0,2025-12-18,Mark A Stevens,Director,S,SELL,222500,180.1680,4.008738e+07
1,2025-12-18,Mark A Stevens,Director,G,OTHER,258650,0.0000,NaN
10,2025-12-17,Jen Hsun Huang,President and CEO,J,OTHER,498131,0.0000,NaN
12,2025-12-17,Jen Hsun Huang,President and CEO,J,OTHER,498131,0.0000,NaN
11,2025-12-17,Jen Hsun Huang,President and CEO,J,OTHER,9141011,0.0000,NaN



📊 STATISTIQUES:
   - Période: 2025-10-21 00:00:00 → 2025-12-19 00:00:00
   - Initiés uniques: 10
   - Achats: 0
   - Ventes: 104
   - Valeur totale des transactions: $304,597,910


In [16]:
# Filtrer les achats haute conviction
if not insider_df.empty:
    print("🎯 Filtrage des achats haute conviction...\n")
    
    high_conviction_df = analyzer.filter_high_conviction_buys(insider_df)
    
    if not high_conviction_df.empty:
        print(f"✅ {len(high_conviction_df)} achats haute conviction détectés\n")
        print(f"🔥 ACHATS HAUTE CONVICTION - {TEST_TICKER}:\n")
        display(high_conviction_df[['transaction_date', 'insider_name', 'role', 'transaction_value', 'conviction_score', 'is_cluster']].head(10))
        
        print(f"\n📊 STATISTIQUES:")
        print(f"   - Valeur totale: ${high_conviction_df['transaction_value'].sum():,.0f}")
        print(f"   - Valeur moyenne: ${high_conviction_df['transaction_value'].mean():,.0f}")
        print(f"   - Score de conviction moyen: {high_conviction_df['conviction_score'].mean():.1f}/100")
        print(f"   - Transactions en cluster: {high_conviction_df['is_cluster'].sum()}")
    else:
        print("⚠️ Aucun achat haute conviction (>$100k) détecté")
else:
    high_conviction_df = pd.DataFrame()

INFO:prod.analysis.edgar_smart_money_analyzer:🎯 Filtering high conviction buys (min $100,000)...
INFO:prod.analysis.edgar_smart_money_analyzer:✅ 0 high conviction buys found


🎯 Filtrage des achats haute conviction...

⚠️ Aucun achat haute conviction (>$100k) détecté


In [ ]:
# Visualisation des transactions d'initiés
if not insider_df.empty:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Distribution des types de transactions
    type_counts = insider_df['type'].value_counts()
    colors = ['green' if t == 'BUY' else 'red' if t == 'SELL' else 'gray' for t in type_counts.index]
    ax1.bar(type_counts.index, type_counts.values, color=colors, edgecolor='black', alpha=0.7)
    ax1.set_title(f'Types de Transactions - {TEST_TICKER}', fontsize=12, fontweight='bold')
    ax1.set_xlabel('Type')
    ax1.set_ylabel('Nombre')
    
    # Distribution des rôles
    role_counts = insider_df['role'].value_counts().head(8)
    ax2.barh(range(len(role_counts)), role_counts.values, color='steelblue', edgecolor='black', alpha=0.7)
    ax2.set_yticks(range(len(role_counts)))
    ax2.set_yticklabels(role_counts.index)
    ax2.set_title(f'Rôles des Initiés - {TEST_TICKER}', fontsize=12, fontweight='bold')
    ax2.set_xlabel('Nombre de Transactions')
    
    plt.tight_layout()
    plt.show()

---

# 🎯 TEST 4: Signaux Combinés

Génération de signaux combinés (Political + Insider) pour tous les tickers

In [ ]:
# Générer les signaux combinés
print(f"🔄 Génération des signaux combinés pour {len(TEST_TICKERS)} tickers...\n")
print("⏳ Cela peut prendre 5-10 minutes (rate limit SEC)\n")

combined_signals_df = analyzer.generate_combined_signals(
    tickers=TEST_TICKERS,
    days_political=60,
    days_insider=30
)

print(f"\n✅ Signaux combinés générés pour {len(combined_signals_df)} tickers")

In [ ]:
# Afficher les signaux combinés
if not combined_signals_df.empty:
    print("🎯 SIGNAUX COMBINÉS (SMART MONEY):\n")
    display(combined_signals_df[['ticker', 'political_score', 'insider_score', 'combined_score', 'recommendation']].sort_values('combined_score', ascending=False))
    
    # Détails supplémentaires
    print("\n📋 DÉTAILS PAR TICKER:\n")
    for _, row in combined_signals_df.iterrows():
        print(f"\n{row['ticker']} - {row['recommendation']} (Score: {row['combined_score']}):")
        if row['details']:
            for key, value in row['details'].items():
                print(f"   - {key}: {value}")
else:
    print("⚠️ Aucun signal combiné généré")

In [ ]:
# Visualisation des signaux combinés
if not combined_signals_df.empty:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Scores combinés par ticker
    sorted_df = combined_signals_df.sort_values('combined_score', ascending=True)
    colors = ['darkgreen' if score >= 70 else 'green' if score >= 50 else 'orange' if score >= 30 else 'gray' 
              for score in sorted_df['combined_score']]
    
    ax1.barh(sorted_df['ticker'], sorted_df['combined_score'], color=colors, edgecolor='black', alpha=0.7)
    ax1.set_title('Scores Combinés Smart Money', fontsize=12, fontweight='bold')
    ax1.set_xlabel('Score Combiné (0-100)')
    ax1.axvline(70, color='red', linestyle='--', alpha=0.5, label='Seuil Très Bullish')
    ax1.axvline(50, color='orange', linestyle='--', alpha=0.5, label='Seuil Bullish')
    ax1.legend()
    
    # Comparaison Political vs Insider scores
    x = np.arange(len(combined_signals_df))
    width = 0.35
    
    ax2.bar(x - width/2, combined_signals_df['political_score'], width, label='Political Score', color='steelblue', alpha=0.7)
    ax2.bar(x + width/2, combined_signals_df['insider_score'], width, label='Insider Score', color='coral', alpha=0.7)
    
    ax2.set_title('Comparaison Political vs Insider Scores', fontsize=12, fontweight='bold')
    ax2.set_xlabel('Ticker')
    ax2.set_ylabel('Score')
    ax2.set_xticks(x)
    ax2.set_xticklabels(combined_signals_df['ticker'])
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

---

# 💾 Export des Résultats

In [ ]:
# Créer le répertoire d'export
export_dir = Path('local_files/smart_money_exports')
export_dir.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Export des transactions politiques
if not political_df.empty:
    filename = export_dir / f'political_trades_{timestamp}.csv'
    political_df.to_csv(filename, index=False)
    print(f"✅ Transactions politiques exportées: {filename}")

# Export des clusters
if not clusters_df.empty:
    filename = export_dir / f'political_clusters_{timestamp}.csv'
    clusters_df.to_csv(filename, index=False)
    print(f"✅ Clusters politiques exportés: {filename}")

# Export des transactions d'initiés
if not insider_df.empty:
    filename = export_dir / f'insider_trades_{TEST_TICKER}_{timestamp}.csv'
    insider_df.to_csv(filename, index=False)
    print(f"✅ Transactions d'initiés exportées: {filename}")

# Export des achats haute conviction
if not high_conviction_df.empty:
    filename = export_dir / f'high_conviction_buys_{TEST_TICKER}_{timestamp}.csv'
    high_conviction_df.to_csv(filename, index=False)
    print(f"✅ Achats haute conviction exportés: {filename}")

# Export des signaux combinés
if not combined_signals_df.empty:
    filename = export_dir / f'combined_signals_{timestamp}.csv'
    combined_signals_df.to_csv(filename, index=False)
    print(f"✅ Signaux combinés exportés: {filename}")

print(f"\n📂 Tous les exports sont dans: {export_dir}")

---

# 📊 Résumé Global

In [ ]:
# Résumé des tests effectués
print("="*70)
print("📊 RÉSUMÉ DES TESTS SMART MONEY")
print("="*70)

print(f"\n1️⃣ TRANSACTIONS POLITIQUES:")
print(f"   - Transactions collectées: {len(political_df) if not political_df.empty else 0}")
print(f"   - Clusters détectés: {len(clusters_df) if not clusters_df.empty else 0}")
print(f"   - Tickers avec activité: {political_df['ticker'].nunique() if not political_df.empty else 0}")

print(f"\n2️⃣ TRANSACTIONS D'INITIÉS ({TEST_TICKER}):")
print(f"   - Transactions collectées: {len(insider_df) if not insider_df.empty else 0}")
print(f"   - Achats haute conviction: {len(high_conviction_df) if not high_conviction_df.empty else 0}")
print(f"   - Valeur totale: ${insider_df['transaction_value'].sum():,.0f}" if not insider_df.empty else "   - Valeur totale: $0")

print(f"\n3️⃣ SIGNAUX COMBINÉS:")
print(f"   - Tickers analysés: {len(combined_signals_df) if not combined_signals_df.empty else 0}")
if not combined_signals_df.empty:
    very_bullish = len(combined_signals_df[combined_signals_df['combined_score'] >= 70])
    bullish = len(combined_signals_df[(combined_signals_df['combined_score'] >= 50) & (combined_signals_df['combined_score'] < 70)])
    interesting = len(combined_signals_df[(combined_signals_df['combined_score'] >= 30) & (combined_signals_df['combined_score'] < 50)])
    
    print(f"   - 🚀 Très Bullish: {very_bullish}")
    print(f"   - 📈 Bullish: {bullish}")
    print(f"   - 💡 Intéressant: {interesting}")
    print(f"\n   Top signal: {combined_signals_df.iloc[0]['ticker']} (Score: {combined_signals_df.iloc[0]['combined_score']})")

print(f"\n4️⃣ EXPORTS:")
print(f"   - Répertoire: {export_dir}")
print(f"   - Timestamp: {timestamp}")

print("\n" + "="*70)
print("✅ Tests terminés avec succès!")
print("="*70)

---

## 💡 Prochaines Étapes

1. **Analyser les exports CSV** pour identifier les patterns
2. **Tester avec d'autres tickers** en modifiant `TEST_TICKERS`
3. **Ajuster les seuils** dans `smart_money_config.py` si nécessaire
4. **Valider les signaux** en comparant avec les mouvements de prix réels
5. **Intégrer au pipeline principal** une fois les tests concluants

---

## 🔧 Dépannage

### Erreurs fréquentes:

**Import Error**: Vérifier que le notebook est dans le répertoire racine `n8n-local-stack`

**SEC Rate Limit**: Attendre 60 secondes et relancer (circuit breaker)

**CIK non trouvé**: Le ticker n'existe pas ou n'est pas référencé par la SEC

**Pas de données**: Normal si aucun Form 4 déposé dans la période

---